# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
Cities_Data = pd.read_csv("output_data/cities.csv")
Cities_Data.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,muhos,0,FI,1647875340,77,64.8080,25.9931,42.58,9.17
1,1,beyneu,26,KZ,1647875341,59,45.3167,55.2000,37.04,14.97
2,2,tasiilaq,0,GL,1647875341,56,65.6145,-37.6368,21.36,3.38
3,3,hermanus,0,ZA,1647875341,67,-34.4187,19.2345,78.80,17.36
4,4,vila franca do campo,64,PT,1647875343,67,37.7167,-25.4333,61.20,16.35


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
locations = Cities_Data[["Lat", "Lng"]].astype(float)
HumidityRate = Cities_Data["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = HumidityRate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#A max temperature lower than 80 degrees but higher than 70, Wind speed less than 10 mph, Zero cloudiness.

Ideal_Weather_Cities_Data = Cities_Data.loc[(Cities_Data["Max Temp"] >= 70) & (Cities_Data["Max Temp"] <= 80) & (Cities_Data["Wind Speed"] <= 10) & (Cities_Data["Cloudiness"] == 0)]
Ideal_Weather_Cities_Data.reset_index(inplace=True)
Ideal_Weather_Cities_Data = Ideal_Weather_Cities_Data.drop(columns=["Cloudiness", "index", "City_ID"])
Ideal_Weather_Cities_Data


,City,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,hilo,US,1647875361,87,19.7297,-155.0900,70.25,3.44
1,yulara,AU,1647875375,27,-25.2406,130.9889,78.64,5.75
2,pimentel,PE,1647875380,69,-6.8367,-79.9342,75.49,5.75
3,cabo san lucas,MX,1647875406,48,22.8909,-109.9124,71.51,5.75
4,sur,OM,1647875408,45,22.5667,59.5289,78.69,9.28
5,port elizabeth,ZA,1647875440,81,-33.9180,25.5701,77.31,5.01
6,umm lajj,SA,1647875538,57,25.0213,37.2685,72.95,6.29
7,pisco,PE,1647875617,78,-13.7000,-76.2167,72.01,5.75


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = Ideal_Weather_Cities_Data
hotel_df["Hotel Name"] = " "
hotel_df

,City,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,hilo,US,1647875361,87,19.7297,-155.0900,70.25,3.44,
1,yulara,AU,1647875375,27,-25.2406,130.9889,78.64,5.75,
2,pimentel,PE,1647875380,69,-6.8367,-79.9342,75.49,5.75,
3,cabo san lucas,MX,1647875406,48,22.8909,-109.9124,71.51,5.75,
4,sur,OM,1647875408,45,22.5667,59.5289,78.69,9.28,
5,port elizabeth,ZA,1647875440,81,-33.9180,25.5701,77.31,5.01,
6,umm lajj,SA,1647875538,57,25.0213,37.2685,72.95,6.29,
7,pisco,PE,1647875617,78,-13.7000,-76.2167,72.01,5.75,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
HotelNames = []

for index, row in hotel_df.iterrows():
    
    lat = hotel_df.loc[index]["Lat"]
    lng = hotel_df.loc[index]["Lng"]

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "type" : "lodging",
        "key": g_key
    }
    
    response = requests.get(base_url, params=params).json()
    results = response["results"]

    try:
        HotelNames.append(results[0]["name"])
        
    except:
        HotelNames.append("")
        
hotel_df["Hotel Name"] = HotelNames
hotel_df

,City,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,hilo,US,1647875361,87,19.7297,-155.0900,70.25,3.44,Hilo Hawaiian Hotel
1,yulara,AU,1647875375,27,-25.2406,130.9889,78.64,5.75,Desert Gardens Hotel - Ayers Rock Resort
2,pimentel,PE,1647875380,69,-6.8367,-79.9342,75.49,5.75,Hotel Puerto del Sol
3,cabo san lucas,MX,1647875406,48,22.8909,-109.9124,71.51,5.75,Hotel Tesoro Los Cabos
4,sur,OM,1647875408,45,22.5667,59.5289,78.69,9.28,Sur Plaza Hotel
5,port elizabeth,ZA,1647875440,81,-33.9180,25.5701,77.31,5.01,39 On Nile Guest House
6,umm lajj,SA,1647875538,57,25.0213,37.2685,72.95,6.29,Moon light Furnished Units
7,pisco,PE,1647875617,78,-13.7000,-76.2167,72.01,5.75,Hotel San Isidro Oficial


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,info_box_content= hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))